In [1]:
import json
import ipywidgets as widgets
import os
import time

In [2]:



# set up the helper routines to load the jsons
# written by J Wallin

def jsonFromFile(fname):
    f = open(fname,"r")
    s = f.read()
    f.close()
    return json.loads(s)

def jsonToFile(fname):
    print(fname)
    
def listFiles(assetPath, printFiles):
    finalList = []
    fullList = os.listdir(assetPath)
    for f in fullList:
        if f.find("json") == len(f)-4:
            print(f)
            finalList.append(f)
    return finalList

def loadFileIntoDictionary(path, flist):
    jdata = {}
    for f in flist:
        jdata[f] = jsonFromFile(path + f)
    return jdata



def writeScene(myScene, verbose):
    # save it to an output file
    outputFile = myScene["jsonFileName"]

    print(os.path.isfile(outputPath + outputFile))

    # if there is a file already there
    print(outputPath + outputFile)
    fileExists = os.path.isfile(outputPath + outputFile)

    if verbose:
        while (fileExists or outputFile == "" ):
    
            print("This is a listing of files in this path: ")
            flist = os.listdir(outputPath)
            for f in flist:
                print(f)
            yn = input("The file " + outputPath + outputFile +" already exists \n Do you want to overwrite it? ")
            if yn == "y" or yn == "Y":
                fileExists = False
            else:
                outputFile = input("What is the new name for our file? ")
                fileExists = os.path.isfile(outputPath + outputFile)
                

    print("\nWriting " + outputPath+outputFile + "\n\n")


    # form the final scene as a string
    myScene["jsonFileName"] = outputFile
    finalJson = json.dumps(myScene, indent=4)
    f = open(outputPath + outputFile, "w", encoding='utf-8')
    f.write(json.dumps(myScene, ensure_ascii=False, indent=4))
    f.close()

    
def createScene(moduleDescription, sceneData, olist, verbose=False):
    sceneData["objects"] = olist
    
    
    # make a copy of the generic scene for the activity file
    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
    myScene.update(moduleDescription)
    myScene.update(sceneData)


    # we can dump the scene to see what it looks like so far  
    if verbose:
        print("This is the Scene\n\n\n")
        print(json.dumps(myScene, indent=4))    
    
    writeScene(myScene, verbose)    
    
    
# this is a code stub for merging two activity files together
def concatenateScenes( outputPath, outputFile, sceneList):

    print(outputPath, outputFile)
    # open the new file
    f = open(outputPath + outputFile, 'w', encoding='utf-8')
    
    for i in range(len(sceneList)):
        print("  \n scene")
        print(i)
        currentScene = sceneList[i]
        print(currentScene)
        
        # read in the files
        fn1 = outputPath + currentScene
        s1 = jsonFromFile(fn1)

        # Dump the file - but do NOT use any indents.  This will flatten
        # the file into a single line.  We then write it out as a line with a newline character.
        a1 = json.dumps(s1, ensure_ascii=False)
        f.write(a1 + "\n")

    # close it
    f.close()
    
def findKeyDiffs(baseClass, olist):
    # find the differences between two dictionaries
    
    # get the keys for the base class
    baseKeys = list(baseClass.keys())
    
    # find the set of common keys for the objects and the set of unique keys
    commonKeys = []
    uniqueKeys = []
    
    # get the key for each object
    for ii in range(len(olist)):
        cc = olist[i]
        objectKeys = list(cc.keys())
    
        # the union of the keys
        uKeys = list(set(baseKeys + objectKeys))
        
        # the intersection of keys
        iKeys = list(set(baseKeys).intersection(objectKeys))
        
        # find the elements only in one list or the other
        aKeys = list( set(baseKeys)^set(objectKeys))

        commonKeys = commonKeys + uKeys
        uniqueKeys = uniqueKeys + aKeys
        
    
    commonKeys = list(set(commonKeys))
    uniqueKeys = list(set(uniqueKeys))
    
    return commonKeys, uniqueKeys
    
        #for ii in range(len(k1)):
        #    k = k1[ii]
        #    if o1[k] != o2[k]:
        #        print(k)
    
    #


def findDiffs(commonKeys, baseClass, olist):
    
    # for each key
    for c in commonKeys:
        baseValue = baseClass[c]
        
        # are there any differences
        diffs = False
        for i in range(len(olist)):
            newValue = olist[i][c]
            if newValue != baseValue:
                diffs = True


        if diffs:
            print("{0:>15}".format(c), end="")
            for i in range(len(olist)):

                if baseValue != olist[i][c]:

                    print("  X   ", end="")
                else:
                    print("  .   ", end="")

            print()
    print(len(olist))
    
#commonKeys, uniqueKeys = findKeyDiffs(earthObject, sortList)   
#findDiffs(commonKeys, earthObject, sortList)


def createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale):
    
    tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }



    position1 = {"x" :topPosition['x'] + lineNumber * offSet['x'],
                "y": topPosition['y'] + lineNumber * offSet['y'],
                "z":topPosition['z'] + lineNumber * offSet['z']
               }

    tdata["textField"] = theText
    textObject = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": position1.copy(),
        "rotation": rotationAngle,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": parentObject,
        "name": textObjectName
    }
    
    return textObject.copy()



def createObjectModification(
    name, # object name
    activationCondition, 
    reactivateObject=False, 
    parentObject="[DYNAMIC]", 
    audioClipName = "",
    goCallback ="",
    newColor= None,
    enable=True, 
    newPosition=None, 
    newEulerAngles=None, 
    newScale=None,
    tmp=None):
    
    
    mm = {"name": name,
         "parentObject": parentObject,
         "activationConditions" : activationCondition,
         "reactiveObject": reactivateObject,
          "enable": enable,
          "position": {"x":0.0, "y":0.0, "z":0.0},
          "newPosition": False,
          "eulerAngles": {"x":0.0, "y":0.0, "z":0.0},
          "newEulerAngles": False,
          "scale": {"x":0.0, "y":0.0, "z":0.0},
          "newScale": False,
             }
                        
    if newPosition != None:
        mm["position"] = newPosition
        mm["newPosition"] = True
            
    if newEulerAngles != None:
        mm["eulerAngles"] = newEulerAngles
        mm["newEulerAngles"] = True
            
    if newScale != None:
        mm["scale"] = newScale
        mm["newScale"] = True
            
    if newColor != None:
        mm['color'] = newColor
    
    if tmp != None:
        mm['tmp'] = tmp
        
    return mm.copy()


def createButton(name, 
                 label, 
                 position, 
                 labelOffset = 0.1,
                 scale = 1, 
                 parentObject = "[_DYNAMIC]",
                 color=[255,0,0,255], 
                 eulerAngles= {"x":90, "y":180.0, "z":0.0}):

    
    scriptDataA = {
    "name": "demoButtonActions",
    "callBackObjects": ["demoModule"]
    }
    scriptDataA = json.dumps(scriptDataA) 


    brbData = {
                "type": "Prefabs/demoButtonPrefab",
                "parentName": parentObject,
                "position": position,
                "eulerAngles": eulerAngles,
                "scale": {"x":scale, "y":scale, "z":scale},
                "name": name, 
                "color": color,
                "componentsToAdd": [scriptDataA]
            }
    
    labelPosition  = {"x":0.0, "y":0.0, "z":labelOffset}
    tdata = {
        "textField": "",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }
    tdata["textField"] = label
    labelScale = 0.6 * scale
    brbLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": labelPosition,
        "eulerAngles": eulerAngles,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": name,
        "name": name+"Label"  
    }
    return brbData, brbLabel


In [3]:
    
# define the path to access the generic templates and the output files
assetPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/genericDefinitions/"     
outputPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/"  
assetPath = "genericDefinitions/"
outputPath = "./"


# get the list of all the files in the generic directory and load them into a dictionary
flist = listFiles(assetPath,  True)
jdata = loadFileIntoDictionary(assetPath, flist)


# this prints out the assets so you can review them
showFiles = False
if (showFiles):
    for f in flist:
        print("===== file name" + f)
        print("---------\n",f, "\n\n\n", json.dumps(jdata[f], indent=4))
        
#print(json.dumps(jdata["genericActivity.json"]))



genericActivity.json
genericObject.json
genericPointerReceiver.json
genericRigidBody.json
genericScriptSimpleRotation.json


In [4]:
# variables for the objects

orbitalScale = 1.0
timeRate = 1.0
earthSize = 0.08
earthRotationTime = 2.0
moonOrbitalPeriod = 56.0
moonOffSetDistance = 2.4
moonSize = earthSize * 0.25


############
scriptData1 = {
    "name": "simpleRotation",
    "timeRate": timeRate,
    "rotationTime": earthRotationTime
}
script1 = json.dumps(scriptData1) 


############
############    
componentsToModify= {
                        "RigidBody": 
                        {
                            "isKinematic": True,
                            "useGravity": False
                        },
                        "PointerReceiver": 
                        {
                        }
                    }

############


earthPosition = { "x":0.0, 
                 "y": 0.0, 
                 "z":1.5}
earthScale = {"x": earthSize,
            "y": earthSize,
            "z": earthSize }

earth = {
                "type": "Prefabs/moveableSphere",
                "position": earthPosition,
                "scale": earthScale,
                "name": "Earth", 
                "parent": "[_DYNAMIC]",
                "texture": "Textures/2k_earth_daymap",
                "componentsToAdd": [script1]
            }


# make a copy of the generic dictionary for objects
earthObject = jdata['genericObject.json'].copy()

# add the generic component attributes
earthObject.update(jdata['genericRigidBody.json'])
earthObject.update(jdata['genericPointerReceiver.json'])
earthObject.update(earth)

# override the generic component attributes
for c in componentsToModify.keys():
    earthObject[c].update(componentsToModify[c])
               
############


moonOffset = {"x":moonOffSetDistance, "y":0.0, "z":0.0}
moonPosition = {"x": earthPosition['x'] + moonOffset['x'],
                "y": earthPosition['y'] + moonOffset['y'],
                "z": earthPosition['z'] + moonOffset['z']
               }

scriptData2 = {
    "name": "simpleOrbit",
    "moonPosition": moonPosition,
    "orbitalPeriod": moonOrbitalPeriod,
    "timeRate": timeRate,
    "orbitalScale": orbitalScale,
    "synchronousRotation":True
    }
script2 = json.dumps(scriptData2)
    
    
moon = {
                "type": "Prefabs/moveableSphere",
                "parent": "[_DYNAMIC]",
                "position": moonPosition,
                "scale": {"x":moonSize, "y":moonSize, "z":moonSize},
                "name": "Moon", 
                "texture": "Textures/_k_moon",
                "componentsToAdd": [script2]
            }



# make a copy of the generic dictionary for objects    
moonObject = jdata['genericObject.json'].copy()

# add the generic component attributes
moonObject.update(jdata['genericRigidBody.json'])
moonObject.update(jdata['genericPointerReceiver.json'])
moonObject.update(moon)

# override the generic component attributes
for c in componentsToModify.keys():
    moonObject[c].update(componentsToModify[c])

############     
############     
   

# this uses a simple prefab
sun = {
        "type": "Prefabs/SunPrefab",
        "position": {"x":-39.0, "y":0, "z":0},
        "scale": {"x":5, "y":5, "z":5},
        "name": "Sun",
        "componentsToAdd": []
    }

# make a copy of the generic dictionary for objects    
sunObject = jdata['genericObject.json'].copy()
sunObject.update(sun)

############ 

tdata = {
        "textField": "8000 miles",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }


labelScale = 10

earthLabelPosition =  {"x":0.0, "y":2, "z":0.0}

tdata["textField"] = "Earth"
earthLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": earthLabelPosition,
        "eulerAngles": {"x": 0.0, "y":0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Earth",
        "name": "earthLabel"
    }


############ 


earthLabelPosition =  {"x":0.0, "y":-2, "z":0.0}


tdata["textField"] = "8000 miles"
labelScale = 0.1
earthSizeLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": earthLabelPosition,
        "eulerAngles": {"x": 0.0, "y":0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Earth",
        "name": "earthSizeLabel"     
    }






############ 



moonLabelPosition = {"x":0.0, "y":4, "z":0.0}

tdata["textField"] = "The Moon"
tdata["fontSize"] = 1
tdata["color"] = [0, 255, 255, 0]
labelScale =20
moonLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "eulerAngles": {"x": 0.0, "y":-90.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonLabel"     
}


############ 


moonLabelPosition = {"x":0.0, "y":-4, "z":0.0}

tdata["textField"] = "2000 miles"
labelScale = 0.1
moonSizeLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "eulerAngles": {"x": 0.0, "y": -90.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonSizeLabel"     
    }


############ 


moonLabelPosition = {"x": 45, "y": 0, "z":0.0}
tdata["textField"] = "240,000 miles"
tdata["fontSize"] = 2.5
labelScale = 0.1
moonDistanceLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "eulerAngles": {"x": 0.0, "y":  180.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonDistanceLabel"     
    }
tdata["fontSize"] = 1


textObjectName = "sectionLabel"
theText = "Sizes and Scales of the Earth and Moon"
fSize=  1.0
lineNumber = 0
topPosition = {"x":0.0, "y": 0.75, "z":2.50}
rotationAngle = {"x": 0.0, "y": 0.0, "z": 0.0}
textColor = [255,255,255,255]
offSet = {"x":0.0, "y":-0.15, "z":0.0}
parentObject = "[_DYNAMIC]"
labelScale = 0.6



sectionLabel = createTextLine( textObjectName, 
               theText, 
               fSize, 
               lineNumber, 
               topPosition, 
               rotationAngle, 
               textColor, 
               offSet, 
               parentObject, 
               labelScale)

####

textObjectName = "taskLabel"
theText = "Introduction"
fSize=  1
lineNumber = 1
labelScale = 0.6



taskLabel = createTextLine( textObjectName, 
               theText, 
               fSize, 
               lineNumber, 
               topPosition, 
               rotationAngle, 
               textColor, 
               offSet, 
               parentObject, 
               labelScale)
 


olist = [sun, earth, moon, earthLabel, earthSizeLabel, moonLabel, moonSizeLabel, moonDistanceLabel, taskLabel, sectionLabel] #, sun, earthSizeLabel, moonLabel, moonSizeLabel]




In [5]:
moon


{'type': 'Prefabs/moveableSphere',
 'parent': '[_DYNAMIC]',
 'position': {'x': 2.4, 'y': 0.0, 'z': 1.5},
 'scale': {'x': 0.02, 'y': 0.02, 'z': 0.02},
 'name': 'Moon',
 'texture': 'Textures/_k_moon',
 'componentsToAdd': ['{"name": "simpleOrbit", "moonPosition": {"x": 2.4, "y": 0.0, "z": 1.5}, "orbitalPeriod": 56.0, "timeRate": 1.0, "orbitalScale": 1.0, "synchronousRotation": true}']}

In [6]:

moduleButtonOffset = 1.2

name = "brbNextModule"
label = "Next\n Module"
labelOffset = 0.13
scale= 1.0

parentObject = "sectionLabel"
position = {"x": moduleButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, labelOffset, scale, parentObject=parentObject)
o1["childColor"] = [0, 255, 255, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)


name = "brbPreviousModule"
label = "Previous \n Module"
labelOffset = 0.13
scale= 1.0
parentObject = "sectionLabel"
position = {"x": -moduleButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, labelOffset, scale=scale, parentObject=parentObject)
o1["childColor"] = [0, 255, 255, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)

##
clipButtonOffset = 1
parentObject = "taskLabel"
labelOffset = 0.13


name = "brbNextClip"
label = "Next"
position = {"x": clipButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, parentObject=parentObject)
o1["childColor"] = [0, 255, 0, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)


name = "brbPreviousClip"
label = "Previous"
position = {"x": -clipButtonOffset, "y": 0, "z":0}
o1, o2 = createButton(name, label, position, parentObject=parentObject)
o1["childColor"] = [0, 255, 0, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)

####





In [7]:
clipList = []
               # 1   2   3   4   5   6   7   8   9  10  11 
timesList2 = [0, 14, -1, -1, 12, 11,  26, -1, 10, 14, 33, -1]
timesList = [0, 14, 10, 9, 12, 11, 26, 10, 14, 33, 9, 19]
taskTitle = ["Introduction", 
             "1 inch= 1000 miles",
             "1 inch= 1000 miles",
             "Moon is 1/4 the Diameter of Earth",
             "Moon is 1/4 the Diameter of Earth",
             "The orbit 30 times Earth's diameter", 
             "The Sun would be about 270 ft away",
            "2 seconds = one day",
            "56 seconds = 4 weeks",
            "Explore the animation","Explore the animation",
            "Work on the handout"]

     #                X   X                               X
     # animate                           X            

        # now that you have seen this demo, it's time to do some work.
        # answer the questions on the handout about the sizes and scales in the Earth Moon system.
        # you can go to previous parts of the demonstration if you need to review the data.
        
clipList = []
for i in range(1, 12): 
    
    if timesList[i] < 0:
        autoAdvance = False
    else:
        autoAdvance = True
        
    autoAdvance = False
    

    clipObj = {"clipName":"clip" + str(i), 
         "audioClipString":"Audio/moonLab/moonLab_1_"+str(i) + "_0",
         "timeToEnd":timesList[i], 
         "autoAdvance": autoAdvance, 
         "objectChanges": [] }
    
    tdata['fontSize'] = 1
    tdata['textField'] = taskTitle[i]
    tdata["parentObject"] =  "[_DYNAMIC]"

    ##tdata['textField'] = "ducky " + str(i)
    oc0a = createObjectModification(
        "taskLabel", 
        0,  
        tmp =json.dumps(tdata)
        )  
    
    clipObj["objectChanges"] = [oc0a]
    clipList.append(clipObj)

oc = {}



'''
def createObjectModification(
    name, # object name
    activationCondition, 
    reactivateObject=False, 
    parentObject="[DYNAMIC]", 
    audioClipName = "",
    goCallback ="",
    newColor= None,
    enable=True, 
    newPosition=None, 
    newEulerAngle=None, 
    newScale=None ):
'''
labelSize = 10
oc1 = createObjectModification("earthSizeLabel", 0,  newScale={"x":labelSize, "y":labelSize, "z":labelSize})
#oc1 = createObjectModification("earthSizeLabel", 0, reactivateObject=True, enable=True)

labelSize = 20
oc2 = createObjectModification("moonSizeLabel", 0, newScale={"x":labelSize, "y":labelSize, "z":labelSize})

labelSize = 30
oc3 = createObjectModification("moonDistanceLabel", 0, newScale={"x":labelSize, "y":labelSize, "z":labelSize})

labelSize = 0.1
oc11a = createObjectModification("earthSizeLabel", 0,  newScale={"x":labelSize, "y":labelSize, "z":labelSize})
oc11b = createObjectModification("moonSizeLabel", 0,  newScale={"x":labelSize, "y":labelSize, "z":labelSize})
oc11c = createObjectModification("moonDistanceLabel", 0, newScale={"x":labelSize, "y":labelSize, "z":labelSize})



clipList[1]["objectChanges"].append(oc1) # = [oc1]
clipList[2]["objectChanges"].append(oc2) #= [oc2]
clipList[3]["objectChanges"].append(oc3) #= [oc3]

clipList[10]["objectChanges"].append(oc11a) #= [oc11a, oc11b, oc11c]
clipList[10]["objectChanges"].append(oc11b) #= [oc11a, oc11b, oc11c]
clipList[10]["objectChanges"].append(oc11c) #= [oc11a, oc11b, oc11c]



    
cmod = createObjectModification(
        "MainInstructions", 
        0
        )

#cmod['canvasText'] = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
#instructions = "In the lab handout, go to the section on Synchronous \n rotation. Explore all the Moon Phases using \nthe Next and Previous buttons.  In the lab handout, \ndraw a stick figure showing the direction \nit is facing on at each phase. "

instructions = """You can navigate through this module by
clicking on the buttons floating above the Earth.  Select
the button with the laser and click on it with the trigger
to activate a button.

Next button - move forward in the module
Previous button - to repeat a previous section

When you have completed the simulation, you will be asked
to answer the questions in the lab handout.  Remember,
you can always go back if you miss some information.

When you are done, click the Red Next Module Button.
"""

cmod['canvasText'] = instructions
clipList[0]['objectChanges'].append(cmod)
    
    
#oc0 = createObjectModification("[_DYNAMIC]", 0,  newPosition={"x":0.0, "y":1.5, "z":0.0})
#clipList[0]["objectChanges"].append(oc0)
    
    
    
    
    

In [8]:
############ button and navigation management




for i in range(len(clipList)):
    

    oc0c = createObjectModification(
        "brbPreviousClip", 
        0 
        )
    

    ###
    oc0d = createObjectModification(
        "brbNextClip", 
        0 
        )    
  
    ###
    oc0e = createObjectModification(
        "brbNextModule", 
        0 
        )    


    
    # previous clip
    if i == 0:
        oc0c["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0c["newScale"] = True     
    else:
        oc0c["scale"] = {"x": 1, "y":1, "z":1}
        oc0c["newScale"] = True     

    # next clip
    if i != len(clipList) -1:
        oc0d["scale"] = {"x": 1, "y":1, "z":1}
        oc0d["newScale"] = True
    else:
        oc0d["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0d["newScale"] = True       

    # next module
    #oc0e["childColor"] = [255, 0, 0, 255]
    #oc0e["childName"] = "button"
    if i == len(clipList) -1:
        oc0e["childColor"] = [255, 0, 0, 255]
        oc0e["childName"] = "button"
        oc0e["scale"] = {"x": 1, "y":1, "z":1}
        oc0e["newScale"] = True  
    else:
        oc0e["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0e["newScale"] = True  
    
    
    clipList[i]["objectChanges"].append(oc0c)
    clipList[i]["objectChanges"].append(oc0d)
    clipList[i]["objectChanges"].append(oc0e)

for c in clipList:
    print( json.dumps(c, indent=4))
    
    
    
    

{
    "clipName": "clip1",
    "audioClipString": "Audio/moonLab/moonLab_1_1_0",
    "timeToEnd": 14,
    "autoAdvance": false,
    "objectChanges": [
        {
            "name": "taskLabel",
            "parentObject": "[DYNAMIC]",
            "activationConditions": 0,
            "reactiveObject": false,
            "enable": true,
            "position": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "newPosition": false,
            "eulerAngles": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "newEulerAngles": false,
            "scale": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "newScale": false,
            "tmp": "{\"textField\": \"1 inch= 1000 miles\", \"color\": [0, 255, 255, 0], \"fontSize\": 1, \"wrapText\": false, \"parentObject\": \"[_DYNAMIC]\"}"
        },
        {
            "name": "M

In [9]:
# The moduleDescription is the text information that is NOT used by the application at runtime.
# It is only data that we will reference in the database of modules and editing.

moduleDescription = {
        "jsonFileName": "moonEarthSizes.json",
        "moduleName": "Sizes and Scales Earth and Moon",
        "description": "A scale model of the Earth and Moon",  
        "author": "John Wallin",
        "authorInstitution": "Middle Tennessee State University",
        "dateCreated": time.strftime("%c")
    }


# MediaInfo[ { "id:", "type": 1}
# MediaURL

# sceneData contains information about prefabs and prefab specific data
sceneData = {
        "prefabName": "demoPrefab",
        "specificName": "Sizes and Scales Earth and Moon",  # this will appear on the navigation window
        "educationalObjectives": ["Students will learn the relative scale of objects in the Earth-Moon System"],
        "instructions": [instructions],
    
        # specific to the demo prefab
        "introAudio": "",
        "createObjects": True,
        "destroyObjects": True,
        "restoreLights": True,
        "timeToEnd": 2000,
        "objects": [],
        "useSunlight": True,
        "clips":clipList
}



createScene(moduleDescription, sceneData, olist, verbose=False)

True
./moonEarthSizes.json

Writing ./moonEarthSizes.json




In [10]:
outputFile = "earthMoonSizes.json"
outputFile = "demo10.json"
#concatenateScenes(outputPath, outputFile, ['basketBallDemo.json', 'miniEarthMoon.json', 'earthMoon.json'])

outputPath = ""
concatenateScenes(outputPath, outputFile, ['moonEarthSizes.json'] )
                  #,'miniEarthMoon.json']  
                  # 'earthMoon.json'])


 demo10.json
  
 scene
0
moonEarthSizes.json


In [11]:
sceneData

{'prefabName': 'demoPrefab',
 'specificName': 'Sizes and Scales Earth and Moon',
 'educationalObjectives': ['Students will learn the relative scale of objects in the Earth-Moon System'],
 'instructions': ['You can navigate through this module by\nclicking on the buttons floating above the Earth.  Select\nthe button with the laser and click on it with the trigger\nto activate a button.\n\nNext button - move forward in the module\nPrevious button - to repeat a previous section\n\nWhen you have completed the simulation, you will be asked\nto answer the questions in the lab handout.  Remember,\nyou can always go back if you miss some information.\n\nWhen you are done, click the Red Next Module Button.\n'],
 'introAudio': '',
 'createObjects': True,
 'destroyObjects': True,
 'restoreLights': True,
 'timeToEnd': 2000,
 'objects': [{'type': 'Prefabs/SunPrefab',
   'position': {'x': -39.0, 'y': 0, 'z': 0},
   'scale': {'x': 5, 'y': 5, 'z': 5},
   'name': 'Sun',
   'componentsToAdd': []},
  {'t

In [12]:
print("done!")

done!
